In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import random

In [3]:
# ============================================
# Step 1: Connect to Qdrant
# ============================================

# Option A: In-memory (no setup needed - easiest!)
client = QdrantClient(":memory:")

# Option B: Connect to local Qdrant server (if using Docker)
# client = QdrantClient(host="localhost", port=6333)

# Option C: Connect to Qdrant Cloud
# client = QdrantClient(url="https://your-cluster.cloud.qdrant.io", api_key="your-api-key")

print("Connected to Qdrant")

Connected to Qdrant


In [4]:
# ============================================
# Step 2: Create a Collection
# ============================================

collection_name = "my_first_collection"

# A collection is like a table in SQL
# We need to specify:
# - size: dimension of vectors (e.g., 5 means each vector has 5 numbers)
# - distance: how to measure similarity (COSINE is most common)

client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=5,  # Each vector will have 5 dimensions
        distance=Distance.COSINE  # Use cosine similarity
    )
)

print(f"Created collection: {collection_name}")

Created collection: my_first_collection


In [5]:
# ============================================
# Step 3: Add Data (Points) to Collection
# ============================================

# In Qdrant, each item is called a "point"
# A point has:
# - id: unique identifier
# - vector: array of numbers
# - payload: metadata (like a JSON object)

# Let's create sample data about fruits
points = [
    PointStruct(
        id=1,
        vector=[0.9, 0.1, 0.1, 0.8, 0.2],  # Vector representing "apple"
        payload={
            "name": "apple",
            "color": "red",
            "category": "fruit",
            "price": 1.5
        }
    ),
    PointStruct(
        id=2,
        vector=[0.9, 0.2, 0.1, 0.7, 0.3],  # Similar to apple
        payload={
            "name": "cherry",
            "color": "red",
            "category": "fruit",
            "price": 3.0
        }
    ),
    PointStruct(
        id=3,
        vector=[0.1, 0.9, 0.2, 0.3, 0.8],  # Different - green fruit
        payload={
            "name": "kiwi",
            "color": "green",
            "category": "fruit",
            "price": 2.0
        }
    ),
    PointStruct(
        id=4,
        vector=[0.2, 0.8, 0.9, 0.2, 0.7],  # Similar to kiwi
        payload={
            "name": "lime",
            "color": "green",
            "category": "fruit",
            "price": 0.5
        }
    ),
    PointStruct(
        id=5,
        vector=[0.5, 0.5, 0.5, 0.5, 0.5],  # Neutral vector
        payload={
            "name": "banana",
            "color": "yellow",
            "category": "fruit",
            "price": 1.0
        }
    )
]

# Insert all points into the collection
client.upsert(
    collection_name=collection_name,
    points=points
)

print(f"Added {len(points)} points to collection")

Added 5 points to collection


In [ ]:
# ============================================
# Step 4: Search for Similar Items
# ============================================

# Let's search for items similar to "apple"
# We use the apple's vector as our query
query_vector = [0.9, 0.1, 0.1, 0.8, 0.2]

search_results = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=3  # Return top 3 similar items
)

print("\nSearch Results (similar to apple):")
print("-" * 50)
for result in search_results:
    print(f"ID: {result.id}")
    print(f"Name: {result.payload['name']}")
    print(f"Color: {result.payload['color']}")
    print(f"Similarity Score: {result.score:.4f}")
    print("-" * 50)



🔍 Search Results (similar to apple):
--------------------------------------------------
ID: 1
Name: apple
Color: red
Similarity Score: 1.0000
--------------------------------------------------
ID: 2
Name: cherry
Color: red
Similarity Score: 0.9901
--------------------------------------------------
ID: 5
Name: banana
Color: yellow
Similarity Score: 0.7643
--------------------------------------------------


C:\Users\snail\AppData\Local\Temp\ipykernel_9296\677047864.py:9: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = client.search(


In [11]:
# ============================================
# Step 5: Search with Filters
# ============================================

# Search for similar items, but only green ones
from qdrant_client.models import Filter, FieldCondition, MatchValue

filtered_results = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    query_filter=Filter(
        must=[
            FieldCondition(
                key="color",
                match=MatchValue(value="green")
            )
        ]
    ),
    limit=3
)

print("\nFiltered Search Results (only green fruits):")
print("-" * 50)
for result in filtered_results:
    print(f"Name: {result.payload['name']}")
    print(f"Color: {result.payload['color']}")
    print(f"Score: {result.score:.4f}")
    print("-" * 50)


Filtered Search Results (only green fruits):
--------------------------------------------------
Name: kiwi
Color: green
Score: 0.3872
--------------------------------------------------
Name: lime
Color: green
Score: 0.3722
--------------------------------------------------


C:\Users\snail\AppData\Local\Temp\ipykernel_9296\1595305384.py:8: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  filtered_results = client.search(


In [12]:
# ============================================
# Step 6: Retrieve Specific Points
# ============================================

# Get a specific point by ID
retrieved = client.retrieve(
    collection_name=collection_name,
    ids=[1, 3]  # Get apple and kiwi
)

print("\nRetrieved Points:")
print("-" * 50)
for point in retrieved:
    print(f"ID: {point.id}")
    print(f"Data: {point.payload}")
    print(f"Vector: {point.vector}")
    print("-" * 50)


Retrieved Points:
--------------------------------------------------
ID: 1
Data: {'name': 'apple', 'color': 'red', 'category': 'fruit', 'price': 1.5}
Vector: None
--------------------------------------------------
ID: 3
Data: {'name': 'kiwi', 'color': 'green', 'category': 'fruit', 'price': 2.0}
Vector: None
--------------------------------------------------


In [14]:
# ============================================
# Step 7: Update a Point
# ============================================

# Update the price of apple
client.set_payload(
    collection_name=collection_name,
    payload={"price": 2.0},  # New price
    points=[1]  # Update point with ID 1 (apple)
)

print("\nUpdated apple's price to 2.0")



Updated apple's price to 2.0


In [15]:
# ============================================
# Step 8: Delete Points
# ============================================

# Delete banana (ID 5)
client.delete(
    collection_name=collection_name,
    points_selector=[5]
)

print("Deleted banana from collection")


Deleted banana from collection


In [20]:
# ============================================
# Step 9: Get Collection Info
# ============================================

collection_info = client.get_collection(collection_name=collection_name)

print("\nCollection Information:")
print("-" * 50)
print(f"Name: {collection_info}")
print(f"Total Points: {collection_info.points_count}")
print(f"Vector Size: {collection_info.config.params.vectors.size}")
print(f"Distance Metric: {collection_info.config.params.vectors.distance}")
print(f"Status: {collection_info.status}")
print("-" * 50)


Collection Information:
--------------------------------------------------
Name: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=4 segments_count=1 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=5, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=None, sharding_method=None, replication_factor=None, write_consistency_factor=None, read_fan_out_factor=None, on_disk_payload=None, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=None, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_c

In [22]:
# ============================================
# Step 10: Scroll Through All Points
# ============================================

# Get all points in the collection
scroll_result = client.scroll(
    collection_name=collection_name,
    limit=10
)

print("\nAll Points in Collection:")
print("-" * 50)
for point in scroll_result[0]:
    print(f"ID: {point.id} - {point.payload['name']}")
print("-" * 50)



All Points in Collection:
--------------------------------------------------
ID: 1 - apple
ID: 2 - cherry
ID: 3 - kiwi
ID: 4 - lime
--------------------------------------------------
